In [1]:
import os
from pathlib import Path
from dotenv import load_dotenv, find_dotenv
import pickle
import asyncio
import nest_asyncio
nest_asyncio.apply()

from src.utils import (
    pdf_data_loader,
    get_nodes,
    get_index,
    get_sentence_window_query_engine,
load_data_to_sql_db,
csv_excel_data_loader,
text_to_query_engine
    )
from sqlalchemy import create_engine
import sqlite3
load_dotenv(find_dotenv())
from llama_index.core import SummaryIndex
from llama_index.core.agent import FunctionCallingAgentWorker, AgentRunner
from llama_index.llms.anthropic import Anthropic
from llama_index.llms.openai import OpenAI
from llama_index.core.selectors import LLMSingleSelector, PydanticSingleSelector
from llama_index.core.query_engine import RouterQueryEngine, SubQuestionQueryEngine
from llama_index.core.tools import QueryEngineTool, ToolMetadata
# Load the autoreload extension
%load_ext autoreload

# Set autoreload to reload all modules automatically before executing code
%autoreload 2

In [29]:
# File Paths
# File Paths
current_path = Path.cwd()
pdf_dirs = current_path.parent / "data" / "pdfs"
# pdf_dirs  = Path("/Users/hamidadesokan/Dropbox/2_Skill_Development/DLML/genai_applications/RAG/data/pdfs")
docs_path = current_path.parent / "data" / "llamadocs"
# docs_path = Path("/Users/hamidadesokan/Dropbox/2_Skill_Development/DLML/genai_applications/RAG/data/llamadocs")
chroma_path = current_path.parent / "data" / "chromadb"
# chroma_path = Path("/Users/hamidadesokan/Dropbox/2_Skill_Development/DLML/genai_applications/RAG/data/chromadb")
sqlitepath = current_path.parent / "data" / "sqlite" / "sqlite.db"
# sqlitepath = Path("/Users/hamidadesokan/Dropbox/2_Skill_Development/DLML/genai_applications/RAG/data/sqlite/sqlite.db")
# Tesla Data
nodes_path = current_path.parent / "data" / "nodes" / "tesla_esg_nodes.pkl"
# nodes_path = Path("/Users/hamidadesokan/Dropbox/2_Skill_Development/DLML/genai_applications/RAG/data/nodes/tesla_esg_nodes.pkl")


# Sales Data
sales_data_path = current_path.parent / "data" / "csv" / "kaggle_sample_superstore.csv"
# sales_data_path = Path("/Users/hamidadesokan/Dropbox/2_Skill_Development/DLML/genai_applications/RAG/data/csv/kaggle_sample_superstore.csv")
sales_nodes_path = current_path.parent / "data" / "nodes" / "sales_nodes.pkl"
# sales_nodes_path = Path("/Users/hamidadesokan/Dropbox/2_Skill_Development/DLML/genai_applications/RAG/data/nodes/sales_nodes.pkl")


parsing_instruction = """
The pdf documents are annual sustainability reports of companies. 
Always quantify your answer using the numbers cited in the relevant text in the document.
Parse every number embedded in the text in a format that is the easiest to use for calculations or retrieval.
Parse the numbers that indicates sustainiability metrics for that year as a list of sustainability metrics with relevanat text and numbers. 
"""


## Load data and save the Document object

In [35]:
fulldocpath = Path.joinpath(docs_path, "10K_esg")
fulllnodespath = Path.joinpath(nodes_path)
def final_query(fulldocpath: Path, fulllnodespath: Path, tablename: str = "Sales" ):
    if fulldocpath.exists():
        print(f"The path {fulldocpath} exists. Loading from disk...")
        with open(fulldocpath, "rb") as f:
            esg_docs = pickle.load(f)
    else:
        print(f"The path {fulldocpath} does not exist. Creating a new one...")
        esg_docs = asyncio.run(pdf_data_loader(pdf_dirs, docs_path, fulldocpath.name, parsing_instruction, 3))
    
    if fulllnodespath.exists():
        print(f"The path {fulllnodespath} exists. Loading from disk...")
        with open(fulllnodespath, "rb") as f:
            esg_nodes = pickle.load(f)
    else:
        print(f"The path {fulllnodespath} does not exist. Creating a new one...")
        esg_nodes = get_nodes(esg_docs, nodes_path, is_markdown=True,
                                        num_workers=6)
    # Create index
    esg_index = get_index(str(chroma_path), fulldocpath.name, esg_nodes)
    summary_esg_index = SummaryIndex(esg_nodes)

    # Create query engine
    esg_query_engine = get_sentence_window_query_engine(esg_index, similarity_top_k=6,
                                                    rerank_top_n=3)

    summary_esg_query_engine = summary_esg_index.as_query_engine(
        response_mode="tree_summarize",
        use_async=True,)

    df = load_data_to_sql_db(str(sales_data_path), str(sqlitepath), tablename)
    conn = sqlite3.connect(str(sqlitepath))
    engine = create_engine("sqlite:///" + str(sqlitepath))
    sales_sql_query_engine = text_to_query_engine([tablename], engine)

    llm = Anthropic(model="claude-3-opus-20240229")
    # llm = OpenAI(temperature=0.1, model= "gpt-3.5-turbo")
    esg_tool = QueryEngineTool(
        query_engine=esg_query_engine,
        metadata=ToolMetadata(
            name = "Tesla10K_Esg_Report_Query_Engine",
            description= "Tool to query McKinsey and Deloitte annual sustainability report. And Teslas 2022 10K financial statement"))
    sql_tool = QueryEngineTool(
        query_engine=sales_sql_query_engine,
        metadata=ToolMetadata(
        name ="SQl engine to query the sqlite sales database",
        description="Useful for translating a natural language query into a SQL query over"
        " a table containing: product information"))
    summary_esg_tool = QueryEngineTool(
        query_engine=summary_esg_query_engine,
        metadata=ToolMetadata(
        name ="Summary ESG Engine",
        description="Useful for summarizing the ESG report"))
    query_engine_tools=[
        esg_tool,
        summary_esg_tool,
        sql_tool]

    ## SUB QUERY
    sq_vec_summary_engine = SubQuestionQueryEngine.from_defaults(
        query_engine_tools=query_engine_tools)
    return sq_vec_summary_engine


In [1]:
from pathlib import Path
from src.utils import  final_query, fulldocpath, fulllnodespath
import asyncio
import nest_asyncio
nest_asyncio.apply()
# fulldocpath = Path.joinpath(docs_path, "10K_esg")
# fulllnodespath = Path.joinpath(nodes_path)
sq_vec_summary_engine = final_query(fulldocpath, fulllnodespath)

The path /Users/hamidadesokan/Dropbox/2_Skill_Development/DLML/genai_applications/RAG/data/llamadocs/10K_esg exists. Loading from disk...
The path /Users/hamidadesokan/Dropbox/2_Skill_Development/DLML/genai_applications/RAG/data/nodes/tesla_esg_nodes.pkl exists. Loading from disk...
loading index 10K_esg


In [30]:
# fulldocpath = Path.joinpath(docs_path, "10K_esg")
# fulllnodespath = Path.joinpath(nodes_path)
# # fullpaths = [fulldocpath, fulllnodespath]
# if fulldocpath.exists():
#     print(f"The path {fulldocpath} exists. Loading from disk...")
#     with open(fulldocpath, "rb") as f:
#         esg_docs = pickle.load(f)
# else:
#     print(f"The path {fulldocpath} does not exist. Creating a new one...")
#     esg_docs = asyncio.run(pdf_data_loader(pdf_dirs, docs_path, fulldocpath.name, parsing_instruction, 3))
#     
# if fulllnodespath.exists():
#     print(f"The path {fulllnodespath} exists. Loading from disk...")
#     with open(fulllnodespath, "rb") as f:
#         esg_nodes = pickle.load(f)
# else:
#     print(f"The path {fulllnodespath} does not exist. Creating a new one...")
#     esg_nodes = get_nodes(esg_docs, nodes_path, is_markdown=True,
#                                         num_workers=6)
# # Create index
# esg_index = get_index(str(chroma_path), fulldocpath.name, esg_nodes)
# summary_esg_index = SummaryIndex(esg_nodes)
# 
# # Create query engine
# esg_query_engine = get_sentence_window_query_engine(esg_index, similarity_top_k=6,
#                                                     rerank_top_n=3)
# 
# summary_esg_query_engine = summary_esg_index.as_query_engine(
#     response_mode="tree_summarize",
#     use_async=True,
# )
# 
# df = load_data_to_sql_db(str(sales_data_path), str(sqlitepath), "Sales")
# conn = sqlite3.connect(str(sqlitepath))
# engine = create_engine("sqlite:///" + str(sqlitepath))
# sales_sql_query_engine = text_to_query_engine(["Sales"], engine)
# 
# llm = Anthropic(model="claude-3-opus-20240229")
# # llm = OpenAI(temperature=0.1, model= "gpt-3.5-turbo")
# esg_tool = QueryEngineTool(
#     query_engine=esg_query_engine,
#     metadata=ToolMetadata(
#         name = "Tesla10K_Esg_Report_Query_Engine",
#          description= "Tool to query McKinsey and Deloitte annual sustainability report. And Teslas 2022 10K financial statement"))
# sql_tool = QueryEngineTool(
#     query_engine=sales_sql_query_engine,
#     metadata=ToolMetadata(
#         name ="SQl engine to query the sqlite sales database",
#         description="Useful for translating a natural language query into a SQL query over"
#         " a table containing: product information"))
# summary_esg_tool = QueryEngineTool(
#     query_engine=summary_esg_query_engine,
#     metadata=ToolMetadata(
#         name ="Summary ESG Engine",
#         description="Useful for summarizing the ESG report"))
# query_engine_tools=[
#         esg_tool,
#         summary_esg_tool,
#         sql_tool]
# 
# ## SUB QUERY
# sq_vec_summary_engine = SubQuestionQueryEngine.from_defaults(
#     query_engine_tools=query_engine_tools)

The path /Users/hamidadesokan/Dropbox/2_Skill_Development/DLML/genai_applications/RAG/data/llamadocs/10K_esg exists. Loading from disk...
The path /Users/hamidadesokan/Dropbox/2_Skill_Development/DLML/genai_applications/RAG/data/nodes/tesla_esg_nodes.pkl exists. Loading from disk...
loading index 10K_esg


In [34]:
fulldocpath.name

'10K_esg'

In [4]:
# esg_docs = asyncio.run(pdf_data_loader(pdf_dirs, docs_path, "10K_esg",parsing_instruction, 3))
with open(Path.joinpath(docs_path, "10K_esg"), "rb") as f:
    esg_docs = pickle.load(f)

In [5]:
# create nodes
# esg_nodes = get_nodes(esg_docs, nodes_path, is_markdown=True,
#                                         num_workers=6)
# 
with open(Path.joinpath(nodes_path), "rb") as f:
    esg_nodes = pickle.load(f)

In [6]:
# Create index
esg_index = get_index(str(chroma_path), "10K_esg", esg_nodes)
summary_esg_index = SummaryIndex(esg_nodes)

loading index 10K_esg


In [7]:
# Create query engine
esg_query_engine = get_sentence_window_query_engine(esg_index, similarity_top_k=6,
                                                    rerank_top_n=3)

summary_esg_query_engine = summary_esg_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)

In [31]:
# esg_query = "How much was Tesla's Research and Development amount and as a percentage of revenue in 2020?"
# esg_query = "How much did McKinsey invested in strengthening risk management since 2018?"
# esg_query = "How much was Tesla's Selling, General, and Administrative expense in 2022?"
esg_query = "How much was Tesla's Selling General and Administrative expense in 2022 as a percentage of revenue?"
# esg_query = "How much did Tesla's Selling, General, and Administrative changed from 2022 vs 2021 in dollar amount? "
response = esg_query_engine.query(esg_query)
print(response)


Tesla's Selling, General and Administrative (SG&A) expenses as a percentage of revenue decreased from 8% in 2021 to 5% in 2022.


In [9]:
df = load_data_to_sql_db(str(sales_data_path), str(sqlitepath), "Sales")
conn = sqlite3.connect(str(sqlitepath))
engine = create_engine("sqlite:///" + str(sqlitepath))
sales_sql_query_engine = text_to_query_engine(["Sales"], engine)

In [10]:
# sales_query ="What is the total sales in Los Angeles"
sales_query ="Which Segment sold the most?" 
response = sales_sql_query_engine.query(sales_query)
print(response)

Based on the SQL query provided, the segment that sold the most is the "Consumer" segment, with a total sales of 1,161,401.34.

The SQL query first groups the sales data by the "Segment" column, calculates the total sales for each segment using the SUM(Sales) function, orders the results by the total sales in descending order, and then takes the first (LIMIT 1) result, which represents the segment with the highest total sales.

Therefore, the response to the given query "Which Segment sold the most?" would be:

"The Consumer segment sold the most, with a total sales of 1,161,401.34."


In [11]:
from llama_index.core.agent import FunctionCallingAgentWorker, AgentRunner
from llama_index.llms.anthropic import Anthropic
from llama_index.llms.openai import OpenAI
from llama_index.core.selectors import LLMSingleSelector, PydanticSingleSelector
from llama_index.core.query_engine import RouterQueryEngine, SubQuestionQueryEngine
from llama_index.core.tools import QueryEngineTool, ToolMetadata
# model_name = "claude-3-haiku-20240307"
llm = Anthropic(model="claude-3-opus-20240229")
# llm = OpenAI(temperature=0.1, model= "gpt-3.5-turbo")
esg_tool = QueryEngineTool(
    query_engine=esg_query_engine,
    metadata=ToolMetadata(
        name = "Tesla10K_Esg_Report_Query_Engine",
         description= "Tool to query McKinsey and Deloitte annual sustainability report. And Teslas 2022 10K financial statement"))
sql_tool = QueryEngineTool(
    query_engine=sales_sql_query_engine,
    metadata=ToolMetadata(
        name ="SQl engine to query the sqlite sales database",
        description="Useful for translating a natural language query into a SQL query over"
        " a table containing: product information"))
summary_esg_tool = QueryEngineTool(
    query_engine=summary_esg_query_engine,
    metadata=ToolMetadata(
        name ="Summary ESG Engine",
        description="Useful for summarizing the ESG report"))
query_engine_tools=[
        esg_tool,
        summary_esg_tool,
        sql_tool]

## ROUTER
# query_engine = RouterQueryEngine(
#     selector=PydanticSingleSelector.from_defaults(llm=llm),
#     query_engine_tools=query_engine_tools,
# )

## SUB QUERY
sq_vec_summary_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools)

## ANTHROPIC AGENT
# agent_worker = FunctionCallingAgentWorker.from_tools(
#     query_engine_tools, llm=llm, verbose=True
# )
# agent = AgentRunner(agent_worker)

In [2]:
# sales_query ="What is the total sales in Los Angeles"
# sales_query ="Which product Segment sold the most using the sales database?"
sales_query ="How much did Tesla's Selling, General, and Administrative changed from 2022 vs 2021 in dollar amount? "
response = sq_vec_summary_engine.query(sales_query)
print(response)

Generated 3 sub questions.
[Tesla10K_Esg_Report_Query_Engine] Q: What was Tesla's Selling, General, and Administrative expenses in 2022?
[Tesla10K_Esg_Report_Query_Engine] Q: What was Tesla's Selling, General, and Administrative expenses in 2021?
[Tesla10K_Esg_Report_Query_Engine] Q: What is the difference in Tesla's Selling, General, and Administrative expenses between 2022 and 2021?
[Tesla10K_Esg_Report_Query_Engine] A: Tesla's Selling, General, and Administrative expenses in 2022 were $3,946 million.
[Tesla10K_Esg_Report_Query_Engine] A: Tesla's Selling, General, and Administrative expenses in 2021 were $4,517 million.
[Tesla10K_Esg_Report_Query_Engine] A: The difference in Tesla's Selling, General, and Administrative (SG&A) expenses between 2022 and 2021 was a decrease of $571 million, or 13%. Specifically, the SG&A expenses were $3,946 million in 2022, compared to $4,517 million in 2021, representing a decrease of $571 million or 13% year-over-year.
The difference in Tesla's Selli

In [5]:
# esg_query = "How is McKinsey responding to the war in Ukraine in its sustainability effort?"
esg_query ="How does Deloitte's sustainbaility goals differs from that of McKinsey"
# esg_query = "Summarize McKinsey ESg report"
# esg_query = "How much was Tesla's Selling, General, and Administrative expense in 2022?"
# esg_query = "How much did Tesla's Selling, General, and Administrative changed from 2022 vs 2021 in dollar amount? "
# esg_query = "How much was Tesla's Selling General and Administrative expense in 2022 as a percentage of revenue?"
response = sq_vec_summary_engine.query(esg_query)
print(response)

Generated 3 sub questions.


RuntimeError: asyncio.run() cannot be called from a running event loop